# Finance & Analytics Club

## CAGR (Compounded Annual Growth Rate)

CAGR is the annual rate of return realized by an asset/portfolio to reach its peak market value from its initial value.

CAGR calculation assumes that the profits are continuously reinvested.

It does not reflect investment rist therefore should always be used in conjunction with a volatility measure.

More at: https://www.investopedia.com/terms/c/cagr.asp

## Annualized Volatility

It is represented by the standard deviation of the returns. Captures the variability of return from the mean return.

It is a widely used method for risk, but does not capture tail risk.

More at: https://www.investopedia.com/terms/v/volatility.asp

## Sharpe Ratio

It is the average return earned in excess of the free risk rate per unit of volatility.

Sharpe ratio greater than 1 is good, greater than 2 is very good and greater than 3 is excellent.

Investors pay close attention to this metric while comparing funds.

More at: https://www.investopedia.com/terms/s/sharperatio.asp

## Sortino Ratio

It is a variation of Sharpe Ration which takes into account standard deviation of only negative returns.

Sharpe Ratio fails to determine the difference between the upward and downward fluctuations, Sortino makes that distinction and therefore considers only harmful volatility.

More at: https://www.investopedia.com/terms/s/sortinoratio.asp

## Maximum Drawdown & Calmar Ratio

Largest price drop in asset price over a specified time period is maximum dropdown.

Investments with longer backtesting period will likely have larger maximum dropdown and therefore caution must be applied in comparing across strategies.

Calmar Ratio is the ration between CAGR and maximum drawdown and it's measure of risk adjusted return.

More at:https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp, https://www.investopedia.com/terms/c/calmarratio.asp

Import necesary libraries

In [5]:
import pandas_datareader.data as pdr
import numpy as np
import datetime as dt

Download historical data for required stocks

In [2]:
ticker = "^GSPC"
SnP = pdr.get_data_yahoo(ticker,dt.date.today()-dt.timedelta(1825),dt.date.today())
SnP

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-04-02,2072.169922,2057.320068,2060.030029,2066.959961,3095960000,2066.959961
2015-04-06,2086.989990,2056.520020,2064.870117,2080.620117,3302970000,2080.620117
2015-04-07,2089.810059,2076.100098,2080.790039,2076.330078,3065510000,2076.330078
2015-04-08,2086.689941,2073.300049,2076.939941,2081.899902,3265330000,2081.899902
2015-04-09,2093.310059,2074.290039,2081.290039,2091.179932,3172360000,2091.179932
...,...,...,...,...,...,...
2020-03-25,2571.419922,2407.530029,2457.770020,2475.560059,8285670000,2475.560059
2020-03-26,2637.010010,2500.719971,2501.290039,2630.070068,7753160000,2630.070068
2020-03-27,2615.909912,2520.020020,2555.870117,2541.469971,6194330000,2541.469971


 ### Function to calculate the Cumulative Annual Growth Rate of a trading strategy

In [3]:
def CAGR(DF):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    df["cum_return"] = (1 + df["daily_ret"]).cumprod()
    n = len(df)/252 # number of years, 252 is number of trading days in a year, use 52/12 if you have weekly/monthly data
    CAGR = (df["cum_return"][-1])**(1/n) - 1
    return CAGR

In [4]:
CAGR = CAGR(SnP)
CAGR

0.04578590378527769

### Function to calculate annualized volatility of a trading strategy

In [6]:
def volatility(DF):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    vol = df["daily_ret"].std() * np.sqrt(252) # use 52/12 if you have weekly/monthly data
    return vol

In [7]:
vol = volatility(SnP)
vol

0.181615767541847

### Function to calculate sharpe ratio ; rf is the risk free rate

In [35]:
def sharpe(DF,rf):
    sr = (CAGR - rf)/vol
    return sr

In [36]:
sharpe = sharpe(SnP, 0.063) # risk free rate in india is about 6.3%
sharpe

-0.09478304911359595

### Function to calculate sortino ratio ; rf is the risk free rate

In [37]:
def sortino(DF,rf):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    neg_vol = df[df["daily_ret"]<0]["daily_ret"].std() * np.sqrt(252)
    sr = (CAGR - rf)/neg_vol
    return sr

In [38]:
sortino  = sortino(SnP, 0.063) # risk free rate in india is about 6.3%
sortino

-0.1040103385704127

### Function to calculate max drawdown

In [39]:
def max_dd(DF):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    df["cum_return"] = (1 + df["daily_ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

In [40]:
mdd = max_dd(SnP)
mdd

0.33924960002653304

### Function to calculate calmar ratio

In [41]:
def calmar(DF):
    clmr = CAGR/mdd
    return clmr

In [42]:
CLMR = calmar(SnP)
CLMR

0.13496229260608333